In [50]:
from numpy import vstack
from numpy import sqrt
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_
from tqdm import tqdm
import torch.nn.functional as F

In [51]:
# dataset definition perparation 
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = read_csv(path, header=None , low_memory=False)
        
        df.head()
        # store the inputs and outputs
        self.X = df.values[1:, 2:6].astype('float32')  # Selecting columns 3 to 6 (indexing is 0-based)
        self.y = df.values[1:, -1].astype('float32')
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

In [52]:
from torch.nn import Module, Linear, ReLU
from torch.nn.init import xavier_uniform_

class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to the first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = ReLU()  # Change to ReLU
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = ReLU()  # Change to ReLU
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
 
    # forward propagate input
    def forward(self, X):
        # input to the first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X


In [53]:
# prepare the dataset
def prepare_data(path ):
    # load the dataset
    dataset = CSVDataset(path)
    
   
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=1024, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl
 
# train the model
def train_model(train_dl, model):
    size = len(train_dl.dataset)
    # define the optimization
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    # enumerate epochs
    for epoch in tqdm(range(30),desc='Training Epochs'):
        print(f"Epoch {epoch+1}\n-------------------------------")
        # enumerate mini batches
        for batch, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
           
            #if batch % 100 == 0:
        loss, current = loss.item(), batch * len(inputs)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
 

In [54]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    return mse

In [55]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [56]:
path = '/kaggle/input/nyse/prices.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(4)
# train the model
train_model(train_dl, model)


570347 280917


Training Epochs:   0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1
-------------------------------


Training Epochs:   3%|▎         | 1/30 [00:08<03:56,  8.15s/it]

loss:     nan  [557668/570347]
Epoch 2
-------------------------------


...

loss:     nan  [557668/570347]
Epoch 29
-------------------------------


Training Epochs:  97%|█████████▋| 29/30 [03:58<00:08,  8.17s/it]

loss:     nan  [557668/570347]
Epoch 30
-------------------------------


Training Epochs: 100%|██████████| 30/30 [04:07<00:00,  8.25s/it]

loss:     nan  [557668/570347]


In [59]:
# # evaluate the model
# mse = evaluate_model(test_dl, model)
# print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))


In [60]:
# make a single prediction R 
# new data 
row = [54.200001 , 53.630001 , 53.389999 , 54.480000]
yhat = predict(row, model)
print('Predicted: %.3f' % yhat)

Predicted: nan
